In [1]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sortedcontainers import SortedList
import math


Tách thành tập train và test


In [2]:
df = pd.read_csv('../dataset/rating_final.csv')
df = shuffle(df)

cut_off_length = int(0.8 * len(df))
df_train = df[:cut_off_length]
df_test = df[cut_off_length:]


In [44]:
user_to_movie = {}
movie_to_user = {}
movie_user_rating = {}

for row in df_train.values:
    movie_id, user_id, rate = row
    if movie_id not in user_to_movie:
        user_to_movie[user_id] = [movie_id]
    else:
        user_to_movie[user_id].append(movie_id)

    if movie_id not in movie_to_user:
        movie_to_user[movie_id] = [user_id]
    else:
        movie_to_user[movie_id].append(user_id)

    movie_user_rating[(movie_id, user_id)] = rate
movie_user_rating

{('tt0060196', 'ur5580339'): 10.0,
 ('tt1831164', 'ur150272282'): 8.0,
 ('tt0099685', 'ur37892465'): 8.7,
 ('tt19623240', 'ur139621299'): 8.0,
 ('tt14986406', 'ur2860723'): 6.0,
 ('tt0110357', 'ur21319675'): 8.5,
 ('tt11245972', 'ur24010163'): 6.3,
 ('tt0071853', 'ur15339371'): 10.0,
 ('tt0089881', 'ur79190944'): 9.0,
 ('tt0141842', 'ur14520497'): 9.2,
 ('tt1832382', 'ur77131575'): 8.3,
 ('tt0077416', 'ur33374263'): 10.0,
 ('tt7322224', 'ur65639948'): 7.3,
 ('tt14986406', 'ur44482045'): 10.0,
 ('tt13833688', 'ur0509769'): 7.7,
 ('tt5555260', 'ur29002389'): 9.0,
 ('tt1454029', 'ur1590118'): 8.1,
 ('tt0107048', 'ur26083589'): 10.0,
 ('tt5753856', 'ur4532636'): 9.4,
 ('tt5074352', 'ur4445210'): 7.0,
 ('tt0472954', 'ur49208568'): 8.9,
 ('tt5687612', 'ur1640094'): 8.7,
 ('tt0068646', 'ur18970655'): 10.0,
 ('tt0071562', 'ur14520497'): 9.0,
 ('tt10332508', 'ur37569728'): 3.0,
 ('tt0077416', 'ur114649661'): 8.0,
 ('tt0073486', 'ur0426613'): 8.7,
 ('tt12758486', 'ur34836174'): 9.0,
 ('tt0099685

In [4]:
movie_user_rating_test = {}
for row in df_test.values:
    movie_id, user_id, rate = row

    movie_user_rating_test[(movie_id, user_id)] = rate
movie_user_rating_test

{('tt0353969', 'ur11927083'): 8.1,
 ('tt2582802', 'ur17822437'): 8.0,
 ('tt3901826', 'ur20327151'): 7.0,
 ('tt0114369', 'ur109135693'): 9.0,
 ('tt1130884', 'ur2721657'): 6.0,
 ('tt0050083', 'ur90868255'): 9.0,
 ('tt0032455', 'ur70682706'): 8.0,
 ('tt0327597', 'ur36494160'): 8.0,
 ('tt0032455', 'ur16488747'): 8.0,
 ('tt0108778', 'ur23650787'): 8.9,
 ('tt0116282', 'ur44376731'): 9.0,
 ('tt10872600', 'ur104763420'): 8.0,
 ('tt0073195', 'ur89337521'): 8.1,
 ('tt0117951', 'ur3153189'): 10.0,
 ('tt0015864', 'ur6587670'): 10.0,
 ('tt0120382', 'ur34756529'): 10.0,
 ('tt2024544', 'ur56393956'): 10.0,
 ('tt0108255', 'ur117926588'): 8.0,
 ('tt0059578', 'ur1980092'): 8.0,
 ('tt0052357', 'ur1293485'): 9.0,
 ('tt0015864', 'ur58619720'): 9.0,
 ('tt0353969', 'ur26443691'): 7.0,
 ('tt5897304', 'ur54207336'): 8.6,
 ('tt11145118', 'ur36922988'): 8.0,
 ('tt0096697', 'ur96996693'): 7.7,
 ('tt0993846', 'ur12009777'): 8.2,
 ('tt0118849', 'ur114700501'): 8.0,
 ('tt0120363', 'ur24085266'): 10.0,
 ('tt1865505',

In [ ]:
user_rating = df_train.pivot_table(index='user_id', columns='movie_id', values='rate', fill_value=0)

In [5]:
number_of_movie = int(len(movie_to_user.keys()))
number_of_user = int(len(user_to_movie.keys()))

print(f'Number of movie: {number_of_movie}')
print(f'Number of user: {number_of_user}')


Number of movie: 490
Number of user: 1000


In [57]:
k_neighbor = 20
limit_neighbor = 5
neighbors = {}
averages = {}

for movie_id_i in movie_to_user.keys():
    # get all user who rated for movie_id_i
    user_rated_movie_i = movie_to_user[movie_id_i]

    # get all rating of users gave for movie_id_i
    rating_user_for_movie_i = [
        movie_user_rating[(movie_id_i, user_id)] for user_id in user_rated_movie_i]

    # calculate avarage rating of all user gave for movie_id_i
    avg_rating_movie_i = np.mean(rating_user_for_movie_i)

    averages[movie_id] = avg_rating_movie_i

    sorted_list = SortedList()
    for movie_id_j in movie_to_user.keys():
        if movie_id_j == movie_id_i:
            continue

        # get all user who rated for movie_id_j
        user_rated_movie_j = movie_to_user[movie_id_j]

        common_users = list(
            set(user_rated_movie_i).intersection(set(user_rated_movie_j)))

        if len(common_users) < limit_neighbor:
            continue

        # get all rating of users gave for movie_id_j
        rating_user_for_movie_j = [
            movie_user_rating[(movie_id_j, user_id)] for user_id in user_rated_movie_j]

        # calculate avarage rating of all user gave for movie_id_j
        avg_rating_movie_j = np.mean(rating_user_for_movie_j)
        
        
        
        deviation = 0
        sigmoid_i = 0
        sigmoid_j = 0
        for user_id in common_users:
            dev_i = (
                movie_user_rating[(movie_id_i, user_id)] - avg_rating_movie_i)
            dev_j = (
                movie_user_rating[(movie_id_j, user_id)] - avg_rating_movie_j)
            deviation += dev_i * dev_j
            sigmoid_i += dev_i * dev_i
            sigmoid_j += dev_j * dev_j

        s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))
        sorted_list.add(s_ij)
        if len(sorted_list) > k_neighbor:
            del sorted_list[0]

    neighbors[movie_id_i] = sorted_list



C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11188\1868976081.py:52: RuntimeWarning: invalid value encountered in double_scalars
  s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))


In [73]:
predictions = {}

for user_id in user_to_movie.keys():
    movies_rated_by_user = user_to_movie[user_id]

    for movie_id in movie_to_user.keys():
        if movie_id in movies_rated_by_user:
            continue

        prediction = 0
        weight_sum = 0

        sorted_list = neighbors[movie_id]
        for neighbor in sorted_list:
            similarity, neighbor_movie_id = neighbor

            if neighbor_movie_id not in movies_rated_by_user:
                continue

            prediction += similarity * (movie_user_rating[(neighbor_movie_id, user_id)] - averages[neighbor_movie_id])
            weight_sum += abs(similarity)

        if weight_sum > 0:
            prediction = prediction/weight_sum + averages[movie_id]
        else:
            prediction = averages[movie_id]

        predictions[(user_id, movie_id)] = prediction


TypeError: cannot unpack non-iterable numpy.float64 object